# Logistic Regression<br>
#### Hypothesis
$$ H(X) = \frac{1}{1+e^{-W^T X}} $$
#### Cost
$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$
If $y \simeq H(x)$, cost is near 0.<br>
If $y \neq H(x)$, cost is high.<br><br>
#### Weight Update via Gradient Descent
$$ W := W - \alpha \frac{\partial}{\partial W} cost(W) $$
$\alpha$: Learning rate

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Data

In [3]:
# x-> (6,2) y -> (6,1)
# 
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6,2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis<br>
* Hypothesis model을 직접 만들어 계산하는 실습 진행

In [6]:
# torch.exp()는 exponential func과 유사하다
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [7]:
# requires_grad는 해당 Tensor에서 행해지는 모든 연산에 대한 미분값 계산 유무이다.
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W)+b)))

In [9]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [10]:
#torch에서 제공하는 sigmoid함수 사용
print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals:  tensor([0.7311])


In [12]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [13]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


torch에서 제공하는 sigmoid함수를 사용해도 결과는 같게 나온다.

## Computing the Cost Function

hypothesis와 y_train간의 차이를 구한다. (H(x) -> P(x=1;W))

In [16]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


직접 cost func을 구현하면

In [18]:
-(y_train[0] * torch.log(hypothesis[0]) +
  (1-y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward>)

In [19]:
#모든 elements에 대해 구현하면
losses = -(y_train * torch.log(hypothesis) + 
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [20]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


* 이를 함수를 통해 구현하면

In [21]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

## Whole Training Procedure

In [45]:
# Model 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# Optimizer 설정 SGD를 통해 W와 b를 training, Learning rate = 1
optimizer = optim.SGD([W, b], lr = 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 계산. grad 초기화, backpropergation 사용, 
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 log 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## Evaluation

In [51]:
#H(x)가 sigmoid를 취한다.
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[0.0242],
        [0.1476],
        [0.2745],
        [0.7957],
        [0.9487]], grad_fn=<SliceBackward>)


In [52]:
#해당 결과의 확률을 가지고 결과를 예측한다
prediction = hypothesis > torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [53]:
#결과를 토대로 정확도를 확인한다
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [54]:
# dataset과 예측 결과의 일치여부를 확인한다.
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


## Higher Implementation with Class

In [84]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8,1) # W
        self.sigmoid = nn.Sigmoid() # b
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [83]:
# model을 BinaryClassifier로 선언
model = BinaryClassifier()

In [82]:
optimizer = optim.SGD(model.parameters(), lr = 1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    
    hypothesis = model(x_train)
    
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0/100 Cost: 0.480745 Accuracy 76.81%
Epoch   10/100 Cost: 0.479340 Accuracy 76.94%
Epoch   20/100 Cost: 0.478214 Accuracy 77.08%
Epoch   30/100 Cost: 0.477303 Accuracy 77.08%
Epoch   40/100 Cost: 0.476556 Accuracy 76.94%
Epoch   50/100 Cost: 0.475939 Accuracy 76.94%
Epoch   60/100 Cost: 0.475424 Accuracy 76.94%
Epoch   70/100 Cost: 0.474990 Accuracy 76.94%
Epoch   80/100 Cost: 0.474622 Accuracy 76.94%
Epoch   90/100 Cost: 0.474307 Accuracy 76.94%
Epoch  100/100 Cost: 0.474036 Accuracy 76.94%
